In [1]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Path where your dataset is saved
data_path = 'data/'

# Image properties
img_size = 128  # Resize images to 128x128

# Empty lists for data
data = []
labels = []

# Categories (folder names)
categories = ['with_mask', 'without_mask']

# Loop through categories
for category in categories:
    folder_path = os.path.join(data_path, category)
    class_label = categories.index(category)  # 0 for with_mask, 1 for without_mask
    
    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        
        # Read image
        img = cv2.imread(img_path)
        if img is None:
            continue  # skip bad files

        # Resize image
        img = cv2.resize(img, (img_size, img_size))
        
        # Normalize image (0-1)
        img = img / 255.0
        
        data.append(img)
        labels.append(class_label)

# Convert to numpy arrays
data = np.array(data)
labels = np.array(labels)

# One-hot encode labels
labels = to_categorical(labels)

# Split into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Data preparation completed successfully!")
print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")


Data preparation completed successfully!
Training samples: 6042, Test samples: 1511


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Model architecture
model = Sequential()

# 1st Convolutional Block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# 2nd Convolutional Block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# 3rd Convolutional Block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=10, 
                    batch_size=32)

# Save the model
model.save("mask_detector_model.h5")

# Evaluate model
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")


c:\Users\bonth\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 69s 346ms/step - accuracy: 0.8115 - loss: 1.8089 - val_accuracy: 0.6671 - val_loss: 1.8848
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 61s 322ms/step - accuracy: 0.8925 - loss: 0.5436 - val_accuracy: 0.6274 - val_loss: 0.8692
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 60s 316ms/step - accuracy: 0.9183 - loss: 0.2849 - val_accuracy: 0.8762 - val_loss: 0.5028
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 60s 319ms/step - accuracy: 0.9434 - loss: 0.1788 - val_accuracy: 0.8504 - val_loss: 1.0155
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 59s 311ms/step - accuracy: 0.9548 - loss: 0.1292 - val_accuracy: 0.9563 - val_loss: 0.1528
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 59s 310ms/step - accuracy: 0.9595 - loss: 0.1085 - val_accuracy: 0.9530 - val_loss: 0.1675
Epoch 7/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 54s 286ms/step - accuracy: 0.9653 - loss: 0.0989 - val_accuracy: 0.9497 - val_loss: 0.1625
Epoch 8/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 57s 302ms/step - accuracy: 0.9771 - loss: 0

48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.9604 - loss: 0.1211
Test Accuracy: 95.90%
